In [3]:
import math
import os
from methods import Segmenter, TransformerModel, LSTM, CRNN, UNet,CCRNN, UNet2, PatchTST
from matplotlib import pyplot as plt
import torch
from utilities import printc, seed
from utils_loader import get_dataloaders, test_idea_dataloader_er_ir, test_idea_dataloader_burpee_pushup
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
from utils_metrics import mean_iou
from torch.nn import functional as F
from train_dense_labeling import get_model
from methods.unet import UNet_encoder
from sklearn.metrics import f1_score 
from transformers import PatchTSTConfig, PatchTSTForClassification
config = {
        'batch_size': 128,
        'epochs':200,
        'fsl': False,
        'model': 'unet',
        'seed': 73054772,
        'dataset': 'physiq'
    }
    
seed(config['seed'])
train_loader, val_loader, test_loader = test_idea_dataloader_er_ir(config)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
patch_config = PatchTSTConfig(
    num_input_channels=6,
    num_targets=2,
    context_length=100,
    patch_length=12,
    stride=12,
    use_cls_token=True,
)
model = PatchTSTForClassification(config=patch_config).float().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# scheduler = ReduceLROnPlateau(optimizer, 'min', patience=10, verbose=True)

for epoch in range(50):
    model.train()
    for batch in tqdm(train_loader):
        x, y = batch
        x, y = x.to(device).float(), y.to(device).long()
        optimizer.zero_grad()
        y_hat = model(past_values=x)
        y_hat = y_hat.prediction_logits
        loss = F.cross_entropy(y_hat, y)
        loss.backward()
        optimizer.step()
    model.eval()
    with torch.no_grad():
        val_loss = 0
        accs = 0

        for batch in val_loader:
            x, y = batch
            x, y = x.to(device).float(), y.to(device).long()
            y_hat = model(past_values=x)
            y_hat = y_hat.prediction_logits
            accs += (y_hat.argmax(1) == y).float().mean()
            val_loss += F.cross_entropy(y_hat, y)
        val_loss /= len(val_loader)
    print(f'Epoch {epoch}, val_loss: {val_loss}')
    print(f'Epoch {epoch}, val_acc: {accs/len(val_loader)}')
    
    # scheduler.step(val_loss)


 33%|███▎      | 1/3 [00:02<00:04,  2.02s/it]


KeyboardInterrupt: 